In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from functools import reduce
import string
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/kyryl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kyryl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
morph_uk = MorphAnalyzer(lang='uk')


def remove_punctuation(text):
    text = text.translate(str.maketrans('', '', string.punctuation + '«»—–-’'))
    return text.replace('\n', ' ').replace(' ', ' ')


def tokenize_text(text):
    words = word_tokenize(text, language='russian')
    return [morph_uk.parse(word)[0].normal_form for word in words]


with open('../data/stopwords_ua.txt') as file:
    ukrainian_stopwords = set(file.read().splitlines())


def remove_stopwords(words):
    return [word for word in words if word not in ukrainian_stopwords]

In [3]:
def transform_text(text):
    return reduce(lambda text, function: function(text),
                  [remove_punctuation, tokenize_text, remove_stopwords], text)

In [4]:
ukr_text_df = pd.read_csv('../data/ukr_text.csv')
ukr_text_df.columns = ukr_text_df.columns.str.lower()
ukr_text_df.head()

,id,title,body
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...


In [5]:
ukr_text_df['words'] = ukr_text_df['body'].apply(transform_text)
ukr_text_df.head()

,id,title,body,words
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве..."
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій..."
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер..."
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г..."
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,..."


In [6]:
tone_df = pd.read_csv('https://raw.githubusercontent.com/lang-uk/tone-dict-uk/master/tone-dict-uk.tsv', delimiter='\t', names=['word', 'tone'], index_col=0)
tone_df.head()

,tone
word,
Всевишній,1
Господь,1
Христовий,1
аборт,-1
абсурд,-1


In [7]:
def calculate_tone(words):
    tone_words = [word for word in words if word in tone_df.index]
    if len(set(tone_words)) < 5:
        return 0
    return sum([tone_df.loc[word].tone if word in tone_df.index else 0 for word in words]) / len(words)

ukr_text_df['tone'] = ukr_text_df['words'].apply(calculate_tone)
ukr_text_df.head()

,id,title,body,words,tone
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве...",0.026163
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій...",-0.090909
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер...",-0.137255
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г...",0.000000
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,...",0.000000


In [8]:
print(ukr_text_df.sort_values(by='tone').iloc[0]['body'])

У Державному департаменті США заявили, що США разом з усім світом згадують жертв Голодомору і вкотре підтвердили прихильність демократії, процвітанню і суверенітету України. Про це йдеться в заяві Держдепу, передає Голос Америки. Прес-секретар Державного департаменту США Морган Ортагюс заявила: "Ми об'єднуємося з усім світом, щоб згадати невинних жертв Голодомору і підтвердити нашу прихильність демократії, процвітанню і суверенітету України". У Держдепі заявили, що Голодомор - одна з найжорстокіших трагедій 20 століття. "Шляхом навмисного захоплення української землі, врожаю і примусової колективізації, Радянський Союз призвів до масштабного голоду, смертей і приніс надзвичайні людські страждання ... Хоча ця жахлива трагедія була однією з найжорстокіших в 20 столітті, Радянському Союзу не вдалося зломити дух українського народу". Раніше повідомлялося, що в Україні відзначають День пам'яті жертв Голодоморів. По всій території України приспущено державні прапори й обмежено проведення зах

In [9]:
print(ukr_text_df.sort_values(by='tone', ascending=False).iloc[0]['body'])

Тільки на мовних курсах у Великій Британії учень з головою може зануритися в природне мовне середовище і отримати безцінний багаж знань. Однак, чим англійська освіта на мовних курсах відрізняється від навчання в інших країнах? Що робить вивчення англійської в Великобританії настільки особливим, що кожен іноземний студент, мріє відправитися на вивчення англійської саме в цю країну? У цій статті, підготувати яку нам допомогло освітнє агентство PFI, ми пропонуємо Вам 5 цікавих фактів про мовні курси в Англії.   Концентрація на розмовних навичках Головною відмінністю мовних курсів в Англії є фокус на розвитку розмовних і комунікативних навичок абітурієнта. Кваліфіковані куратори груп планують навчальний процес таким чином, щоб теоретична частина займала 20%, а практика - 80%. Подібний підхід до навчання вперше застосували навчальні заклади Великобританії, раніше з'ясувавши, що активне обговорення і постійна мовна практика підвищують загальну успішність студента і результативність всього ку

In [10]:
ukr_text_df['normalized_text'] = ukr_text_df.words.str.join(' ')
ukr_text_df

,id,title,body,words,tone,normalized_text
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве...",0.026163,перспективний відкритий ринок бізнес інвестиці...
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій...",-0.090909,зявитися відео конфлікт мотоцикліст водій авто...
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер...",-0.137255,бессарабський площа вранці четвер 16 серпень в...
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г...",0.000000,вночі 16 серпень пережити найсильніший гроза з...
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,...",0.000000,уночі вкотре накрити негода найсильніший дощ п...
...,...,...,...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...,"[київський, офіс, класичний, ювелірний, дома, ...",0.042616,київський офіс класичний ювелірний дома лоборт...
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...,"[20, том, орден, заслуга, називатися, почесний...",0.124682,20 том орден заслуга називатися почесний знак ...
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...,"[10, жовтень, політичний, важковаговик, табір,...",0.022042,10 жовтень політичний важковаговик табір опози...
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т...","[ізраїль, країна, займати, місце, світ, надій,...",0.018657,ізраїль країна займати місце світ надій молоко...


In [11]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(ukr_text_df['normalized_text'])

In [12]:
svd_vectorizer = TruncatedSVD(n_components=100, random_state=42)
X_lsa = svd_vectorizer.fit_transform(X)

In [13]:
X_lsa

array([[ 0.22554268, -0.01841248,  0.07070091, ..., -0.02838924,
         0.01491171,  0.00846253],
       [ 0.09406398, -0.02790445, -0.0320689 , ...,  0.07981946,
         0.01932493, -0.04900203],
       [ 0.09417164, -0.02188734, -0.03898791, ...,  0.04596805,
         0.07591701, -0.02015498],
       ...,
       [ 0.20440434, -0.0483453 , -0.00387152, ..., -0.02651493,
        -0.01794904, -0.06926745],
       [ 0.21970905, -0.02814167,  0.03929289, ..., -0.01025367,
        -0.03825015, -0.04493127],
       [ 0.24004565, -0.07530466, -0.05597721, ..., -0.04029973,
         0.03659069, -0.1107553 ]])

In [14]:
def get_category_words(vectorizer, svd, n_top_words):
    words = vectorizer.get_feature_names_out()
    topics = []
    for component in svd.components_:
        top_words_idx = np.argsort(component)[::-1][:n_top_words]
        top_words = [words[i] for i in top_words_idx]
        topics.append(top_words)
    return topics

In [15]:
categories = get_category_words(tfidf_vectorizer, svd_vectorizer, 10)
for i, category in enumerate(categories):
    print(f'Words in category {i}: {", ".join(category)}')

Words in category 0: україна, рок, долар, компанія, росія, новий, йога, країна, область, грудень
Words in category 1: нафта, біржа, долар, індекс, барель, пункт, марка, торг, ньюйоркський, brent
Words in category 2: область, регіон, інвестиція, газа, млрд, грн, населення, душити, транзит, україна
Words in category 3: долар, газа, транзит, росія, україна, нафтогаз, газпром, нафта, російський, барель
Words in category 4: індекс, пункт, jones, dow, nasdaq, фондовий, 500, виборчий, ринок, цвк
Words in category 5: виборчий, округа, цвк, комісія, депутат, народний, серпень, обраний, зареєструвати, голос
Words in category 6: гонка, область, кубок, україна, регіон, збірний, підручний, естафета, змішаний, інвестиція
Words in category 7: матч, динамо, штрафний, мяч, шахтар, пробити, ворота, очко, команда, збірний
Words in category 8: мерседес, феррарі, феттля, хемілтон, гранпрі, боттас, себастьян, булла, льюїс, хаас
Words in category 9: шоу, учасник, випуск, мастершеф, сезон, санкція, північний,

In [16]:
ukr_text_df['category'] = X_lsa.argmax(axis=1)
ukr_text_df

,id,title,body,words,tone,normalized_text,category
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве...",0.026163,перспективний відкритий ринок бізнес інвестиці...,0
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій...",-0.090909,зявитися відео конфлікт мотоцикліст водій авто...,0
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер...",-0.137255,бессарабський площа вранці четвер 16 серпень в...,21
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г...",0.000000,вночі 16 серпень пережити найсильніший гроза з...,57
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,...",0.000000,уночі вкотре накрити негода найсильніший дощ п...,0
...,...,...,...,...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...,"[київський, офіс, класичний, ювелірний, дома, ...",0.042616,київський офіс класичний ювелірний дома лоборт...,0
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...,"[20, том, орден, заслуга, називатися, почесний...",0.124682,20 том орден заслуга називатися почесний знак ...,0
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...,"[10, жовтень, політичний, важковаговик, табір,...",0.022042,10 жовтень політичний важковаговик табір опози...,0
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т...","[ізраїль, країна, займати, місце, світ, надій,...",0.018657,ізраїль країна займати місце світ надій молоко...,0
